In [ ]:
import numpy as np
import pandas as pd
#import fiona 
import geopandas as gpd
#import math
#from netCDF4 import Dataset
#import xarray as xa
import matplotlib.pyplot as plt
%matplotlib inline
#from matplotlib import cm
#from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import datetime
import random
from scipy.interpolate import interp1d

In [ ]:
# File written by dday_slope-radadj.ipynb
radadj_fn = 'c:/Users/markstro/work1.1/soltab/dday_slope_radadj_GF_v1.1.csv'
radadj_solf_fn = 'c:/Users/markstro/work1.1/soltab/dday_slope_radadj_solf_GF_v1.1.csv'
pptadj_fn = 'c:/Users/markstro/work1.1/soltab/dday_slope_pptadj_GF_v1.1.csv'
gdb_path = 'c:/Users/markstro/work1.1/GIS/GFv1.1_v2e.gdb'

param_path = "c:/Users/markstro/work1.1/paramdb_v1.1/paramdb_master/"

In [ ]:
nhru_v11 = gpd.read_file(gdb_path, layer='nhru_v11')

In [ ]:
nhru_v11_ids = nhru_v11["nhru_v11"].values
nhru = len(nhru_v11_ids)
print(nhru)
print(nhru_v11_ids)

nmonths = 12
print(nhru * nmonths)

In [ ]:
# Read PRMS parameters
tmax_allrain_offset_f = pd.read_csv(param_path + "tmax_allrain_offset.csv")
tmax_allrain_offset = tmax_allrain_offset_f["tmax_allrain_offset"].values

tmax_allsnow_f = pd.read_csv(param_path + "tmax_allsnow.csv")
tmax_allsnow = tmax_allsnow_f["tmax_allsnow"].values
tmax_allrain = tmax_allsnow + tmax_allrain_offset

#print(tmax_allrain)
#print(tmax_allrain.shape)

tmax_allrain.shape = (nmonths, nhru)

#print(tmax_allrain.shape)

dday_intcp_f = pd.read_csv(param_path + "dday_intcp.csv")
dday_intcp = dday_intcp_f["dday_intcp"].values
dday_intcp.shape = (nmonths, nhru)
#print(dday_intcp)

# These 2D parameters are all set to constants so there is only one value
ppt_rad_adj = 0.02
tmax_index = 50.0
radj_sppt = 0.44
radj_wppt = 0.55
radadj_intcp = 1.0
radadj_slope = 0.02

In [ ]:
# read the radadj values from the csv file produced by dday_slope-radadj.ipynb
radadj_df = pd.read_csv(radadj_fn)

In [ ]:
radadj = radadj_df.iloc[:, 1:367].values
print(radadj.shape)
print(radadj)

In [ ]:
radadj_nhru_v11_v = radadj_df["nhru_v11"].values
print(radadj_nhru_v11_v)

# Need to compute pptadj with pptadj_fn not random

In [ ]:
# read the pptadj values from the csv file produced by dday_slope-pptadj.ipynb
# pptadj_df = pd.read_csv(pptadj_fn)

dates = [datetime.datetime(2015, 1, 1), datetime.datetime(2015, 1, 2), datetime.datetime(2015, 1, 3),
         datetime.datetime(2015, 1, 4)]
ndays = len(dates)

foo = np.random.random_sample(((nhru*ndays),))
foo.shape = (4, nhru)

hru_id_str = np.empty(nhru, dtype='int')
for jj in range(nhru):
    hru_id_str[jj] = jj+1

# Create the pandas DataFrame. This should be comparable to what has been produced and read in from pptadj_fn)
pptadj_df = pd.DataFrame(foo, columns = hru_id_str)
pptadj_df["date"] = dates

pptadj_df.head()

In [ ]:
pptadj_v = pptadj_df.iloc[:, 0:nhru].values
print(pptadj_v)

In [ ]:
radadj_solf = np.zeros((len(pptadj_df["date"]), nhru))

ii = 0
for date in pptadj_df["date"]:
    jday = date.dayofyear
    print(date, jday)
    
    for ihru in range(nhru):
        radadj_solf[ii,ihru] = radadj[ihru,jday] / pptadj_v[ii,ihru]
    
    ii += 1


In [ ]:
print(radadj_solf.shape)
print(radadj_solf)

In [ ]:
# Create the pandas DataFrame with the radadj values. This is for writing a csv file.
radadj_solf_df = pd.DataFrame(radadj_solf, columns = nhru_v11_ids)
radadj_solf_df["date"] = dates

# This moves the "dates" column to the first position, better for the CSV.
cols = list(radadj_solf_df.columns)
cols = [cols[-1]] + cols[:-1]
radadj_solf_df = radadj_solf_df[cols]

radadj_solf_df.to_csv(path_or_buf=radadj_solf_fn)

In [ ]:
radadj_solf_df.head()

In [ ]:
solf_pts = np.array([.20, .35, .45, .51, .56, .59, .62, .64, .655, .67, .682, .69, .70, .71, .715, .72, .722, .724, .726,
      .728, .73, .734, .738, .742, .746, .75])
print(solf_pts)
dday_pts = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0,
                     18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0])
print(type(solf_pts))
f2 = interp1d(solf_pts, dday_pts, kind='cubic')

print(f2(0.7))

In [ ]:
xnew = np.linspace(0.201, 0.749, num=41, endpoint=False)

#for ii in range(len(xnew)):
#    print(xnew[ii])
#    print(xnew[ii], f2(xnew[ii]))

plt.plot(xnew,f2(xnew))
plt.show()

In [ ]:
dday = np.zeros(radadj_solf.shape)

ii = 0
for date in pptadj_df["date"]:
    jday = date.dayofyear
    print(date, jday)
    
    for ihru in range(nhru):
        if radadj_solf[ii, ihru] < 0.2:
            dday[ii,ihru] = 1.0
        elif radadj_solf[ii, ihru] > 0.75:
            dday[ii,ihru] = 26.0
        else:
            dday[ii,ihru] = f2(radadj_solf[ii,ihru])
    
    ii += 1

In [ ]:
print(dday)

# Need to read tmax_hru from the CBH ncf file

In [ ]:
# dday = Dday_slope(j, Nowmonth)*Tmax_hru(j) + Dday_intcp(j, Nowmonth) + 1.0
# dday_slope(ihru,imon) = (dday(iday,ihru) - dday_intcp(ihru,imon) - 1.0) / tmax_hru(iday,ihru)

ii = 0
for date in pptadj_df["date"]:
    jday = date.dayofyear
    imon = date.month - 1
    print(date, jday, imon)
    
    for ihru in range(nhru):
        dday_slope_daily[ihru,ii] = (dday[ii,ihru] - dday_intcp[imon,ihru] - 1.0) / tmax_hru[ii,ihru]

In [ ]:
for date in pptadj_df["date"]:
    jday = date.dayofyear
    imon = date.month
    print(date, jday, imon)
    
    for ihru in range(nhru):
        dday_slope[ihru,imon] += dday_slope_daily[ihru,ii]
        count[imon] += 1
        
for ihru in range(nhru):
    for ihru in range(0,12):
        dday_slope[ihru,imon] = dday_slope[ihru,imon] / count[imon]
